In [74]:
import torch
from ttt.agents.az.training.trainer import AlphaZeroTrainer
from ttt.agents.az.training import load_config_from_yaml
from ttt.agents.az.net import masked_softmax
from ttt.env.state import TTTEnv
import torch.nn.functional as F

In [84]:
config_path = "ttt/ttt_config.yaml"
config = load_config_from_yaml(config_path)
trainer = AlphaZeroTrainer(config)
trainer.load_checkpoint('ttt/runs/run_20251030_112736/checkpoints/alphazero_epoch_25.pt')

Saved checkpoint: checkpoints\alphazero_epoch_0.pt
Loaded checkpoint from ttt/runs/run_20251030_112736/checkpoints/alphazero_epoch_25.pt


c:\Users\occoo\UTTT\ttt\agents\az\training\trainer.py:523: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=self.config.de

25

In [121]:
env = TTTEnv()
env.step(4)
env.step(6)
env.step(8)
env.step(1)
env.step(7)
obs_tensor = torch.from_numpy(env._encode()).unsqueeze(0).float().to('cuda')
net = trainer.network
logits, value = net.forward(obs_tensor)

legal_mask = env.legal_actions_mask()
legal_mask_tensor = torch.from_numpy(legal_mask.flatten()).unsqueeze(0).bool().to('cuda')

priors = F.softmax(logits, dim=-1) if legal_mask_tensor is None else masked_softmax(logits, legal_mask_tensor, dim=-1)
logits, priors, value

(tensor([[ 0.1809, -3.9389,  0.0949,  0.5206, -3.7663,  4.7562, -1.8865, -4.5252,
          -2.5813]], device='cuda:0', grad_fn=<AddmmBackward0>),
 tensor([[0.0100, 0.0000, 0.0091, 0.0140, 0.0000, 0.9669, 0.0000, 0.0000, 0.0000]],
        device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor([0.0090], device='cuda:0', grad_fn=<SqueezeBackward1>))

In [77]:
trainer_2 = AlphaZeroTrainer(config)
trainer_2.load_checkpoint('ttt/runs/run_20251029_213207/checkpoints/alphazero_epoch_0.pt')

env_2 = TTTEnv()
obs_tensor = torch.from_numpy(env_2._encode()).unsqueeze(0).float().to('cuda')
net_2 = trainer_2.network
logits, value = net_2.forward(obs_tensor)
legal_mask = env.legal_actions_mask()
legal_mask_tensor = torch.from_numpy(legal_mask.flatten()).unsqueeze(0).bool().to('cuda')

priors = F.softmax(logits, dim=-1) if legal_mask_tensor is None else masked_softmax(logits, legal_mask_tensor, dim=-1)
logits, priors, value

Saved checkpoint: checkpoints\alphazero_epoch_0.pt
Loaded checkpoint from ttt/runs/run_20251029_213207/checkpoints/alphazero_epoch_0.pt


(tensor([[ 0.2103,  0.1144, -0.6679, -0.2835, -0.1678, -0.6790,  0.4892, -0.4699,
          -0.7762]], device='cuda:0', grad_fn=<AddmmBackward0>),
 tensor([[0.1605, 0.1458, 0.0667, 0.0979, 0.1099, 0.0659, 0.2121, 0.0813, 0.0598]],
        device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor([0.7452], device='cuda:0', grad_fn=<SqueezeBackward1>))